In [1]:
# reset all variables
%reset -f

In [2]:
import requests
import zipfile
import io
import pandas as pd
import xml.etree.ElementTree as ET
import os

In [3]:
# post_ridgecrest_schema_local_filename = r'C:\Users\lblair\Documents\Data\SCEC\post_ridgecrest_schema.csv'
post_ridgecrest_schema_local_filename = 'post_ridgecrest_schema.csv'

### Napa observation table

In [4]:
# --- fetch Napa XML metadata from ScienceBase ---
item_id = '5c1a392de4b0708288c2e9a5'
sb_json_url = f'https://www.sciencebase.gov/catalog/item/{item_id}?format=json'
# sb_json_url = f'https://www.sciencebase.gov/catalog/item/5c1a392de4b0708288c2e9a5?format=json'
response = requests.get(sb_json_url)
data = response.json()

# Find the XML URL
xml_url = None
for file in data.get("files", []):
    if file["name"] == "Observations.xml":
        xml_url = file["url"]
        break

# Load and parse XML
if xml_url:
    xml_response = requests.get(xml_url)
    napa_xml_root = ET.fromstring(xml_response.content)

    # Extract schema definitions from XML
    def extract_schema_from_xml(root):
        records = []
        for attr in root.iter():
            if attr.tag.endswith('attr'):
                name = attr.findtext('.//attrlabl')
                definition = attr.findtext('.//attrdef')
                if name:
                    records.append({
                        'Field': name.strip(),
                        'Definition': definition.strip() if definition else ''
                    })
        return pd.DataFrame(records)

    napa_schema_df = extract_schema_from_xml(napa_xml_root)

    # Load Observations.txt into DataFrame
    txt_url = next((f["url"] for f in data.get("files", []) if f["name"] == "Observations.txt"), None)
    if txt_url:
        napa_df = pd.read_csv(txt_url, sep='\t')

        # Merge schema with actual columns
        napa_merged_schema = pd.DataFrame({'Field': napa_df.columns})
        napa_merged_schema = pd.merge(napa_merged_schema, napa_schema_df, on='Field', how='left')
        napa_merged_schema.head(25)
    else:
        print("Observations.txt not found.")
else:
    print("Observations.xml not found.")


In [5]:
napa_df.head(5)

,stnid,intid,citation,obs_date,latitude,longitude,orig_lat,orig_lon,photo,observer,...,description,fault_azimuth,ss_displacement,ss_sense,ext_offset,comp_offset,vert_offset,upthrown_side,trace,origin
0,1.0,161,NaN,2014:09:11,38.205764,-122.309377,38.205902,-122.309395,Y,DeLong,...,Minor cracking; no obvious offset; possibly ol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,Uncertain
1,2.0,790,NaN,2014:08:26,38.206033,-122.299086,38.206028,-122.299070,Y,Lancaster,...,Mole tracks along levee. Trend 320 azimuth. Ab...,320,NaN,NaN,NaN,NaN,NaN,NaN,A,Uncertain
2,3.0,791,NaN,2014:08:26,38.206754,-122.300993,38.206795,-122.300972,Y,Lancaster,...,Northeast trending fractures through levee. 2-...,10,NaN,NaN,NaN,NaN,NaN,NaN,A,Uncertain
3,4.0,792,NaN,2014:08:26,38.207986,-122.305738,38.208069,-122.305672,Y,Lancaster,...,Faint mole track in levee. Roughly 4 in wide w...,310,NaN,NaN,NaN,NaN,NaN,NaN,A,Uncertain
4,5.0,163,NaN,2014:09:11,38.208184,-122.309187,38.208134,-122.309158,Y,DeLong,...,Cracking. Compiler note: On UAVSAR lineament t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,Tectonic


In [6]:
# Print the schema
napa_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222 entries, 0 to 1221
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   stnid             1221 non-null   float64
 1   intid             1222 non-null   int64  
 2   citation          84 non-null     object 
 3   obs_date          1222 non-null   object 
 4   latitude          1222 non-null   float64
 5   longitude         1222 non-null   float64
 6   orig_lat          1220 non-null   float64
 7   orig_lon          1220 non-null   float64
 8   photo             1222 non-null   object 
 9   observer          1222 non-null   object 
 10  observed_feature  1222 non-null   object 
 11  description       811 non-null    object 
 12  fault_azimuth     177 non-null    object 
 13  ss_displacement   166 non-null    object 
 14  ss_sense          243 non-null    object 
 15  ext_offset        131 non-null    object 
 16  comp_offset       12 non-null     object 


In [7]:
# column definitions
with pd.option_context('display.max_colwidth', None):
    display(napa_merged_schema)

,Field,Definition
0,stnid,"Identifier for the observation station. A station is assigned where descriptions, displacement measurements and (or) photographs have been obtained at a site at a given date and time by an observer or group of observers working together. Repeat observations at different times by the same observer or group of observers are assigned different stnids."
1,intid,Internal identifier. Used to link an observation record to associated records in the Photos table.
2,citation,"If the data or photographs from this observation are published elsewhere, the original publication is cited here."
3,obs_date,"Date of the observation as year, numerical month, and day of month."
4,latitude,"Latitude coordinate of the station in decimal degrees, WGS84 datum. These values are derived either from the coordinate provided directly by the observer, or are digitized during compilation using airborne imagery and ground photographs to more accurately locate the station on the feature of interest. Observations of fault features were typically relocated to coincide with mapped fault locations as derived from high-resolution imagery, lidar data or InSAR data."
5,longitude,"Longitude coordinate of the station in decimal degrees, WGS84 datum. These values are derived either from the coordinate provided directly by the observer, or are digitized during compilation using airborne imagery and ground photographs to more accurately locate the station on the feature of interest. Observations of fault features were typically relocated to coincide with mapped fault locations as derived from high-resolution imagery, lidar data or InSAR data."
6,orig_lat,"Original latitude reported by the observer in decimal degrees, WGS84 datum."
7,orig_lon,"Original longitude reported by the observer in decimal degrees, WGS84 datum."
8,photo,Indicates if photographs are associated with this station
9,observer,Last name of the observer or observers who provided the information/photographs. Full names and affiliations are listed in the data set credit of this metadata record. A single name may be listed where observations may have been collected by a group of individuals working together.


In [8]:
print(f'number of Napa observations: {len(napa_df)}')

number of Napa observations: 1222


### Ridgecrest Observation Table

In [9]:
# Step 1: ScienceBase item ID and JSON metadata URL
item_id = '5ef18f7f82ced62aaae19257'
sb_json_url = f'https://www.sciencebase.gov/catalog/item/{item_id}?format=json'

# Step 2: Get metadata from ScienceBase
response = requests.get(sb_json_url)
data = response.json()

# Step 3: Find the ZIP file and XML metadata file URLs
zip_url = None
xml_url = None
for file in data.get('files', []):
    if file['name'] == 'CSV_File_Ridgecrest_Observations_Slip_Prov_Rel_1.zip':
        zip_url = file['url']
    elif file['name'] == 'Ridgecrest_Observations_Slip_Prov_Rel_1.xml':
        xml_url = file['url']

# Step 4: Download and extract the CSV file
if zip_url and xml_url:
    zip_response = requests.get(zip_url)
    with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
        csv_name = next(
            name for name in z.namelist()
            if name.endswith('.csv') and not name.startswith('__MACOSX')
        )
        with z.open(csv_name) as csvfile:
            df_ridgecrest = pd.read_csv(csvfile)

    # Step 5: Download and parse the XML file
    xml_response = requests.get(xml_url)
    ridgecrest_xml_root = ET.fromstring(xml_response.content)

    # Step 6: Extract schema from XML
    def extract_schema_from_xml(root):
        records = []
        for attr in root.iter():
            if attr.tag.endswith('attr'):
                name = attr.findtext('.//attrlabl')
                definition = attr.findtext('.//attrdef')
                if name:
                    records.append({
                        'Field': name.strip(),
                        'Definition': definition.strip() if definition else ''
                    })
        return pd.DataFrame(records)

    ridgecrest_schema_df = extract_schema_from_xml(ridgecrest_xml_root)

    # Step 7: Merge schema with DataFrame columns
    merged_ridgecrest_schema = pd.DataFrame({'Field': df_ridgecrest.columns})
    merged_ridgecrest_schema = pd.merge(merged_ridgecrest_schema, ridgecrest_schema_df, on='Field', how='left')

    pd.set_option('display.max_colwidth', None)
    merged_ridgecrest_schema.head(25)
else:
    print("ZIP or XML file not found.")

In [10]:
df_ridgecrest.head(5)

,intid,origid,observer,obs_affiliation,team_id,team,obs_position,obs_date,origin,source,...,vert_slip_type,heave_type,heave_min,heave_pref,heave_max,latitude,longitude,orig_lat,orig_lon,note
0,19,19,G. Funning,UCR,'07/04-2',G. Funning/UCR,At Site,2019-07-04T19:37:17-07,Tectonic,6.4,...,NaN,NaN,NaN,NaN,NaN,35.644039,-117.536706,35.644039,-117.536706,Measurement estimated from photograph.
1,22,Red Point 3,C. Madugo,PG&E,'07/05-5',C. Hitchcock/InfraTerra\nO. Kozaci/InfraTerra\nC. Madugo/PG&E,At Site,2019-07-05T00:00:00-07,Tectonic,6.4,...,NaN,NaN,NaN,NaN,NaN,35.623983,-117.567485,35.623983,-117.567485,Observation time not reported
2,28826,1,I. Pierce,UNR,'07/05-4',B. Brooks/USGS\nR. Koehler/UNR\nI. Pierce/UNR,At Site,2019-07-05T07:38:29-07,Tectonic,6.4,...,NaN,Extension,NaN,60.0,NaN,35.639533,-117.540619,35.639533,-117.540619,NaN
3,28827,2,I. Pierce,UNR,'07/05-4',B. Brooks/USGS\nR. Koehler/UNR\nI. Pierce/UNR,At Site,2019-07-05T07:41:44-07,Tectonic,6.4,...,NaN,Extension,NaN,30.0,NaN,35.639899,-117.539949,35.639899,-117.539949,NaN
4,28829,4,I. Pierce,UNR,'07/05-4',B. Brooks/USGS\nR. Koehler/UNR\nI. Pierce/UNR,At Site,2019-07-05T09:00:32-07,Tectonic,6.4,...,Separation,Extension,NaN,NaN,1.0,35.637147,-117.544223,35.637147,-117.544223,NaN


In [11]:
# Print the schema
df_ridgecrest.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130 entries, 0 to 1129
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   intid                   1130 non-null   int64  
 1   origid                  1088 non-null   object 
 2   observer                1130 non-null   object 
 3   obs_affiliation         1130 non-null   object 
 4   team_id                 1130 non-null   object 
 5   team                    1129 non-null   object 
 6   obs_position            1130 non-null   object 
 7   obs_date                1130 non-null   object 
 8   origin                  1130 non-null   object 
 9   source                  1129 non-null   object 
 10  citation                1 non-null      object 
 11  description             1067 non-null   object 
 12  fault_az_min            86 non-null     float64
 13  fault_az_pref           724 non-null    float64
 14  fault_az_max            90 non-null     

In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(ridgecrest_schema_df)


,Field,Definition
0,intid,Observation identifier assigned at compilation
1,origid,Original name or ID of observation station as defined by the observer
2,observer,The principal team member making the observation.
3,obs_affiliation,The agency or organization affiliated with the principal observer.
4,team_id,Field team identifier
5,team,Team members
6,obs_position,The location of the observer relative to the actual deformation.
7,obs_date,"The date and time that the observation was made. A textual description of the date\nas numeric values of the year, month, day separated by dashes. Time of observation is\npresented after the letter ""T"" with hours, minutes, seconds each separated by a colon and\nfollowed by a time zone offset. For example, 2019-07-05T17:44:55-07 represents July 5,\n2019, at 5:44 PM and 55 seconds, with a time zone offset of -7 hours before Greenwich Mean\nTime."
8,origin,"An enumerated value describing the inferred origin of the observed feature, based\non ground observations and other factors such as rupture continuity and surface\nexpression."
9,source,"The earthquake presumed to have produced the observed deformation (M6.4 - July 4),\n(M7.1 - July 5), or Uncertain"


In [13]:
print(f'number of Ridgecrest observations: {len(df_ridgecrest)}')

number of Ridgecrest observations: 1130


### Post-Ridgecrest Schema

In [14]:
# Read CSV into DataFrame
df_current_schema = pd.read_csv(post_ridgecrest_schema_local_filename)
df_current_schema

,Unnamed: 0,Field Name,Field Type,Alias
0,0,OBJECTID,esriFieldTypeOID,OBJECTID
1,1,Station_ID,esriFieldTypeString,"Station ID (Initials_sequential number (i.e., TED_001))"
2,2,Feature_Origin,esriFieldTypeString,Feature
3,3,Notes,esriFieldTypeString,Notes
4,4,Confidence_Feature_ID,esriFieldTypeString,Confidence in Feature ID
5,5,Mode_Observation,esriFieldTypeString,Mode of Observation
6,6,Slip_Sense,esriFieldTypeString,Slip Sense
7,7,Scarp_Facing_Direction,esriFieldTypeString,Scarp Facing Direction
8,8,Local_Fault_Dip,esriFieldTypeSmallInteger,Local Fault Dip (0-90°; right hand rule)
9,9,Local_Fault_Azimuth_Degrees,esriFieldTypeSmallInteger,Local Fault Azimuth Degrees (0-360°; right hand rule)


In [ ]:
# # pull directly from AGOL, save as csv

from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# # Connect to ArcGIS Online or your Enterprise portal
# # gis = GIS("home")  # Use "home" if you're already logged in
# gis = GIS("https://www.arcgis.com", "my_username", "my_password")

# # Replace with your item ID
# item_id = "4659de32b8514b34bf842321b05c0e63"  # <- your item ID here

# # Get the item and its feature layer(s)
# item = gis.content.get(item_id)
# flc = FeatureLayerCollection.fromitem(item)
# layer = flc.layers[0]  # Or loop if you want all layers

# # Create the DataFrame
# schema_df = pd.DataFrame([
#     {
#         "Field Name": field["name"],
#         "Field Type": field["type"],
#         "Alias": field.get("alias", "")
#     }
#     for field in layer.properties.fields
# ])

# # Display the DataFrame
# display(schema_df)

# schema_df.to_csv(post_ridgecrest_schema_local_filename)

Exception: You do not have permissions to access this resource or perform this operation.
(Error Code: 403)

In [ ]:
# # Fixed complete schema comparison across all three datasets

# # Define the helper function first
# def get_current_field_meaning(field_name):
#     """Get the meaning/alias for a specific field name in current schema"""
#     row = df_current_schema[df_current_schema['Field Name'] == field_name]
#     if not row.empty:
#         return row['Alias'].iloc[0]
#     else:
#         return f"Field '{field_name}' not found"

# # Create standardized schema comparison across all three datasets
# current_schema_standardized = df_current_schema[['Field Name', 'Alias']].copy()
# current_schema_standardized.columns = ['Field', 'Definition']
# current_schema_standardized['Dataset'] = 'Current (Post-Ridgecrest)'

# # Add dataset labels to existing schemas
# napa_schema_labeled = napa_merged_schema.copy()
# napa_schema_labeled['Dataset'] = 'Napa (2014)'

# ridgecrest_schema_labeled = ridgecrest_schema_df.copy()
# ridgecrest_schema_labeled['Dataset'] = 'Ridgecrest (2019)'

# # Combine all schemas
# all_schemas = pd.concat([
#     napa_schema_labeled[['Field', 'Definition', 'Dataset']], 
#     ridgecrest_schema_labeled[['Field', 'Definition', 'Dataset']], 
#     current_schema_standardized[['Field', 'Definition', 'Dataset']]
# ], ignore_index=True)

# print("Schema Evolution Summary:")
# print("=" * 25)
# print(f"Napa (2014): {len(napa_schema_labeled)} fields")
# print(f"Ridgecrest (2019): {len(ridgecrest_schema_labeled)} fields")  
# print(f"Current (Post-Ridgecrest): {len(current_schema_standardized)} fields")

# # Find common fields across datasets
# napa_fields = set(napa_schema_labeled['Field'].dropna())
# ridgecrest_fields = set(ridgecrest_schema_labeled['Field'].dropna())
# current_fields = set(current_schema_standardized['Field'].dropna())

# # Core fields present in all three schemas
# common_all_three = napa_fields & ridgecrest_fields & current_fields
# print(f"\nFields common to all three schemas: {len(common_all_three)}")
# if common_all_three:
#     print("Common fields:", sorted(list(common_all_three)))

# # Fields unique to each dataset
# current_only = current_fields - napa_fields - ridgecrest_fields
# ridgecrest_only = ridgecrest_fields - napa_fields
# napa_only = napa_fields - ridgecrest_fields - current_fields

# # Find overlaps between datasets
# napa_ridgecrest_common = napa_fields & ridgecrest_fields
# current_ridgecrest_common = current_fields & ridgecrest_fields
# current_napa_common = current_fields & napa_fields

# print(f"\nDataset Overlaps:")
# print(f"Napa-Ridgecrest common: {len(napa_ridgecrest_common)}")
# print(f"Current-Ridgecrest common: {len(current_ridgecrest_common)}")
# print(f"Current-Napa common: {len(current_napa_common)}")

# print(f"\nUnique to each dataset:")
# print(f"Unique to Napa only: {len(napa_only)}")
# print(f"New in Ridgecrest (not in Napa): {len(ridgecrest_only)}")
# print(f"New in Current (not in Napa or Ridgecrest): {len(current_only)}")

# # Analyze field categories in current schema
# def categorize_current_fields():
#     """Categorize current schema fields by purpose"""
#     categories = {
#         'Identification': [],
#         'Location': [],
#         'Observation_Metadata': [],
#         'Fault_Geometry': [],
#         'Slip_Measurements': [],
#         'Confidence_Quality': [],
#         'Secondary_Effects': [],
#         'System_Fields': []
#     }
    
#     for _, row in df_current_schema.iterrows():
#         field = row['Field Name']
#         alias = str(row['Alias']).lower()
#         field_lower = field.lower()
        
#         if any(word in field_lower for word in ['id', 'station']):
#             categories['Identification'].append(field)
#         elif any(word in alias for word in ['latitude', 'longitude', 'coordinate', 'location']):
#             categories['Location'].append(field)  
#         elif any(word in alias for word in ['observer', 'date', 'time', 'mode']) or 'observation' in alias:
#             categories['Observation_Metadata'].append(field)
#         elif any(word in alias for word in ['fault', 'dip', 'azimuth', 'strike', 'scarp']):
#             categories['Fault_Geometry'].append(field)
#         elif any(word in alias for word in ['slip', 'displacement', 'separation', 'offset', 'heave']):
#             categories['Slip_Measurements'].append(field)
#         elif any(word in alias for word in ['confidence', 'quality', 'notes']):
#             categories['Confidence_Quality'].append(field)
#         elif any(word in alias for word in ['landslide', 'liquefaction', 'damage', 'slope']):
#             categories['Secondary_Effects'].append(field)
#         elif any(word in field_lower for word in ['globalid', 'objectid', 'creation', 'edit']):
#             categories['System_Fields'].append(field)
#         else:
#             # Default to observation metadata for uncategorized
#             categories['Observation_Metadata'].append(field)
    
#     return categories

# categories = categorize_current_fields()

# print("\nCurrent Schema Field Categories:")
# print("=" * 35)
# for category, fields in categories.items():
#     if fields:  # Only show categories with fields
#         print(f"\n{category.replace('_', ' ')} ({len(fields)} fields):")
#         for field in fields:
#             meaning = get_current_field_meaning(field)
#             print(f"  - {field}: {meaning}")

# # Create a comparison table for key measurement fields
# key_measurements = ['horizontal', 'vertical', 'slip', 'offset', 'displacement', 'azimuth']

# def find_measurement_fields(schema_df, field_col='Field'):
#     """Find measurement-related fields in a schema"""
#     measurement_fields = []
#     for _, row in schema_df.iterrows():
#         field = str(row[field_col]).lower()
#         if any(keyword in field for keyword in key_measurements):
#             measurement_fields.append(row[field_col])
#     return measurement_fields

# print("\nMeasurement Field Evolution:")
# print("=" * 30)
# napa_measurements = find_measurement_fields(napa_schema_labeled)
# ridgecrest_measurements = find_measurement_fields(ridgecrest_schema_labeled)
# current_measurements = find_measurement_fields(current_schema_standardized)

# print(f"Napa measurement fields ({len(napa_measurements)}): {napa_measurements}")
# print(f"Ridgecrest measurement fields ({len(ridgecrest_measurements)}): {ridgecrest_measurements}")  
# print(f"Current measurement fields ({len(current_measurements)}): {current_measurements}")

# # Show some specific field comparisons
# print("\nField Name Evolution Examples:")
# print("=" * 35)

# # Look for similar concepts across schemas
# concept_mappings = {
#     'Station/Site ID': ['stnid', 'intid', 'Station_ID'],
#     'Observer Info': ['observer', 'observer', 'Creator'],
#     'Fault Azimuth': ['fault_azimuth', 'fault_az_pref', 'Local_Fault_Azimuth_Degrees'],
#     'Horizontal Offset': ['ss_displacement', 'horiz_offset_pref', 'Horizontal_Separation_cm'],
#     'Vertical Offset': ['vert_offset', 'vert_offset_pref', 'Vertical_Separation_cm']
# }

# for concept, field_names in concept_mappings.items():
#     print(f"\n{concept}:")
#     datasets = ['Napa', 'Ridgecrest', 'Current']
#     for i, (dataset, field_name) in enumerate(zip(datasets, field_names)):
#         print(f"  {dataset}: {field_name}")

Schema Evolution Summary:
Napa (2014): 21 fields
Ridgecrest (2019): 63 fields
Current (Post-Ridgecrest): 57 fields

Fields common to all three schemas: 0

Dataset Overlaps:
Napa-Ridgecrest common: 11
Current-Ridgecrest common: 0
Current-Napa common: 0

Unique to each dataset:
Unique to Napa only: 10
New in Ridgecrest (not in Napa): 52
New in Current (not in Napa or Ridgecrest): 57

Current Schema Field Categories:

Identification (11 fields):
  - OBJECTID: OBJECTID
  - Station_ID: Station ID (Initials_sequential number (i.e., TED_001))
  - Confidence_Feature_ID: Confidence in Feature ID
  - Rupture_Width_m: Rupture Width Preferred (m)
  - Rupture_Width_Min_m: Rupture Width Min (m)
  - Rupture_Width_Max_m: Rupture Width Max (m)
  - Vector_Measurement_Confidence: Vector Measurement Confidence
  - Slip_Measurement_Confidence: Slip Measurement Confidence
  - Landslide_Feature: Landslide Feature
  - Slide_Type: Slide Type
  - GlobalID: GlobalID

Observation Metadata (17 fields):
  - Feature